In [1]:
%cd ..

/misc/vlgscratch4/LakeGroup/wentao/multimodal-baby


In [2]:
import itertools
import functools
from collections import namedtuple
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
from multimodal.multimodal_lit import MultiModalLitModel
from multimodal.multimodal_data_module import PAD_TOKEN_ID, UNK_TOKEN_ID, SOS_TOKEN_ID, EOS_TOKEN_ID
from ngram import NGramModel
from multimodal.utils import map_structure
from analysis_tools.processing import examples_from_dataloader, get_pos_tags, get_model_items, ModelItems, run_model_on_data_batches
from analysis_tools.sumdata import *
from analysis_tools.build_data import build_data
from analysis_tools.pos_tags import *
from analysis_tools.utils import *
from analysis_tools.word_categories import *


figsize = (13, 12)
sns.set_theme(
    style='white',
    font='monospace',
    rc={
        'figure.figsize': figsize,
        'axes.grid': False,
        'xtick.bottom': False,
        'xtick.top': False,
        'ytick.left': False,
        'ytick.right': False,
        'axes.spines.left': False,
        'axes.spines.bottom': False,
        'axes.spines.right': False,
        'axes.spines.top': False,
    }
)

np.set_printoptions(suppress=True, precision=2, linewidth=120)
pd.options.display.width = 120

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pos_mapping = pos_mappings[2][1]

used_poses = ["noun", "verb"]

/usr/local/stow/python-3.9/lib/python3.9/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [3]:
def pos_tagged_seq_repr(tokens, pos_tags):
    return ' '.join(f'{token}/{pos}' for token, pos in zip(tokens, pos_tags))


class Template(namedtuple('TemplateTuple', ['seq', 'pos', 'idx'])):
    """A template where the token at idx of seq is to be filled.
    """

    def __str__(self):
        return ' '.join(
            '_' * len(token) if i == self.idx else token
            for i, (token_id, token) in enumerate(
                (token_id, idx2word[token_id]) for token_id in self.seq)
            if token_id not in [SOS_TOKEN_ID, EOS_TOKEN_ID])


def templates_from_example(token_ids, pos_tags):
    for i, (token_id, pos) in enumerate(zip(token_ids, pos_tags)):
        if pos_mapping[pos] in used_poses:
            yield Template(token_ids, pos_tags, i)


def templates_from_examples(examples, pos_tags, print_pos_tagged_seq=False):
    tot, cnt_filtered = 0, 0

    for example, y_pos_tags in zip(examples, pos_tags):
        x, y, y_len, raw_y = example
        tot += 1
        if print_pos_tagged_seq:
            print(pos_tagged_seq_repr(raw_y[0].split(), y_pos_tags[1:]))
        cnt_present_tokens = sum((pos_mapping[pos] not in ["."] for pos in y_pos_tags))
        if cnt_present_tokens <= 2:
            continue
        y_len = y_len.item()
        y = y[:y_len].tolist()
        if UNK_TOKEN_ID in y:
            continue

        cnt_filtered += 1
        yield example, templates_from_example(y, y_pos_tags)

    print(f'filtered {cnt_filtered} / {tot} = {cnt_filtered / tot :.2%} examples')


def joined_templates_from_examples(*args, **kwargs):
    for example, example_templates in templates_from_examples(*args, **kwargs):
        for template in example_templates:
            yield example, template


def construct_batch_by_filling_template(x, template, word_ids):
    batch_size = len(word_ids)
    x_batch = x.unsqueeze(0).expand(batch_size, *([-1] * x.dim()))
    y = torch.tensor(template.seq)
    y_batch = y.unsqueeze(0).repeat(batch_size, *([1] * y.dim()))
    y_batch[:, template.idx] = torch.tensor(word_ids)
    y_len = torch.tensor(len(template.seq))
    y_len_batch = y_len.unsqueeze(0).expand(batch_size, *([-1] * y_len.dim()))
    raw_y_batch = [[]] * batch_size
    return x_batch, y_batch, y_len_batch, raw_y_batch


def run_model_on_template(model, example, template, word_ids, batch_size=256):
    """Run model and get the whole sentence losses on template filled with every word_id in word_ids
    Inputs:
        model: the model
        example: the example; need to provide this for the model to have image x
        template: the template
        word_ids: fill the template with every word_id in word_ids
        batch_size: the batch size
    Returns:
        losses: an np.ndarray of length len(word_ids)
    """

    x, y, y_len, raw_y = example
    x = x.to(device=device)

    batches = (construct_batch_by_filling_template(x, template, word_ids[i : i + batch_size])
               for i in range(0, len(word_ids), batch_size))
    losses = []
    for batch in run_model_on_data_batches(model, batches, use_tqdm=False):
        outputs, loss = batch[4:]
        loss = loss.sum(-1)
        losses.append(loss.detach())
    losses = torch.cat(losses, 0)
    return losses


def find_index(lst, e):
    for i, x in enumerate(lst):
        if x == e:
            return i
    return None


def get_prob_dist(model, example, template, word_ids, word_cats, batch_size=256, top_k=None, words=None):
    """Run model on template and get the probability distribution of categories;
    check whether model gives higher probability to the correct category.
    Inputs:
        model: the model
        example: the example; need to provide this for the model to have image x
        template: the template
        word_ids: fill the template with every word_id in word_ids
        word_cats: np.ndarray of same length as word_ids; the categories of each word in word_ids
        batch_size: the batch size used to run the model
        top_k: print top k predictions along with the ground-truth label; None if unwanted
        words: words correspond to words_ids; used in printing
    Returns:
        pd.Series with categories indexing the probability distribution
    """

    losses = run_model_on_template(model, example, template, word_ids, batch_size=batch_size)
    probs = F.softmax(-losses, 0)
    if top_k is not None:
        used_word_ids = word_ids
        used_words = words
        used_losses = losses
        used_probs = probs
        gt_word_id = template.seq[template.idx]
        gt_word_idx = find_index(used_word_ids, gt_word_id)
        if gt_word_idx is None:  # the ground-truth word is not in the vocab of prediction
            print('ground-truth word not in the vocab of prediction')
            # append the ground-truth word to the vocab
            gt_word_idx = len(word_ids)
            used_word_ids = np.append(used_word_ids, gt_word_idx)
            used_words = np.append(used_words, idx2word[gt_word_id])
            used_losses = torch.cat([used_losses, torch.tensor([np.inf], dtype=used_losses.dtype, device=used_losses.device)])
            used_probs = torch.cat([used_probs, torch.tensor([0.], dtype=used_probs.dtype, device=used_probs.device)])
        print_top_values(-used_losses, used_words, labels=torch.tensor(gt_word_idx), value_formatter=lambda loss: f'{-loss:6.3f}')
        print_top_values(used_probs, used_words, labels=torch.tensor(gt_word_idx), value_formatter=prob_formatter)
    probs_ser = pd.Series(probs.cpu())
    return probs_ser.groupby(word_cats).sum()

In [4]:
# build data and vocab according to the model
data, args = build_data(return_args=True)
dataset_name = args.dataset
vocab = data.datasets['train'].vocab
vocab_size = len(vocab)
print(f'vocab_size = {vocab_size}')
word2idx = vocab
idx2word = [None] * vocab_size
for word, idx in word2idx.items():
    idx2word[idx] = word
idx2word = np.array(idx2word)

my_batch_size = 256
dataloader_fns = {
    'train': lambda: data.train_dataloader(batch_size=my_batch_size, shuffle=False, drop_last=False),
    'val': lambda: data.val_dataloader(batch_size=my_batch_size)[0],
    'test': lambda: data.test_dataloader(batch_size=my_batch_size)[0],
}

checkpoint_path = "checkpoints/lm_dataset_saycam_captioning_False_text_encoder_lstm_embedding_dim_512_dropout_i_0.5_dropout_o_0.0_batch_size_16_lr_0.006_lr_scheduler_True_weight_decay_0.04_seed_0/epoch=29.ckpt"
print(f"load model from {checkpoint_path}")
model = MultiModalLitModel.load_from_checkpoint(checkpoint_path, map_location=device)
model.to(device)

# get the pos tags of all words in vocab
# by the index of model_items produced by get_model_items
train_dataloader_fn = dataloader_fns['train']
train_pos_tags = get_pos_tags(train_dataloader_fn(), dataset_name, 'train')
train_model_items = torch_cache(checkpoint_path + f'.train.cache.pt')(get_model_items)(
    model, train_dataloader_fn(), train_pos_tags, ignore_all_token_items=True)
idx2pos = np.full(vocab_size, '.', dtype='O')
for token_id, pos in train_model_items.token_items.index:
    idx2pos[token_id] = pos
idx2pos2 = np.vectorize(pos_mapping.get)(idx2pos)
for pos in sorted(set(pos_mapping.values())):
    pos_word_ids = np.nonzero(idx2pos2 == pos)[0]
    print(f'#{pos:15s}: {len(pos_word_ids)}')
idx2used = np.isin(idx2pos2, used_poses)
if True:
    idx2used[UNK_TOKEN_ID] = False
word_ids = np.nonzero(idx2used)[0]
words = idx2word[word_ids]
word_cats = idx2pos2[word_ids]

/home/ww2135/.local/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:73: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


Using base transforms
Calling prepare_data!
SAYCam transcripts have already been downloaded. Skipping this step.
Transcripts have already been renamed. Skipping this step.
Transcripts have already been preprocessed. Skipping this step.
Training frames have already been extracted. Skipping this step.
Training metadata files have already been created. Skipping this step.
Evaluation frames have already been extracted. Skipping this step.
Evaluation metadata files have already been created. Skipping this step.
Evaluation metadata files have already been created. Skipping this step.
Extra evaluation metadata files have already been created. Skipping this step.
Vocabulary file already exists. Skipping this step.
Calling setup!
vocab_size = 2350
load model from checkpoints/lm_dataset_saycam_captioning_False_text_encoder_lstm_embedding_dim_512_dropout_i_0.5_dropout_o_0.0_batch_size_16_lr_0.006_lr_scheduler_True_weight_decay_0.04_seed_0/epoch=29.ckpt
load cached pos tags: dataset_cache/saycam/t

In [5]:
splits = ['val']

my_untypical_words = ' '.join(
    possessives + negations + be_verbs + do_verbs + modal_verbs +
    pronoun_contractions + other_contractions + quantifiers + pos_ambiguous_words +
    special_tokens
).split()
filter_out_untypical_words = False

for split in splits:
    dataloader_fn = dataloader_fns[split]
    examples = examples_from_dataloader(dataloader_fn())
    pos_tags = get_pos_tags(dataloader_fn(), dataset_name, split)
    golds, preds, corrects = [], [], []
    for example, template in joined_templates_from_examples(examples, pos_tags):
        if filter_out_untypical_words and (idx2word[template.seq[template.idx]] in my_untypical_words):
            continue
        print(template)
        gold = pos_mapping[template.pos[template.idx]]
        prob_dist = get_prob_dist(model, example, template, word_ids, word_cats, top_k=5, words=words)
        print(prob_dist)
        argmax_idx = prob_dist.argmax()
        pred = prob_dist.index[argmax_idx]
        correct = pred == gold
        print(f'{gold = }, {pred = }, {correct = }')
        golds.append(gold)
        preds.append(pred)
        corrects.append(correct)
    print(f'label distribution: ')
    frac_format = lambda a, b, prec=2: f'{a} / {b} = {a/b:.{prec}%}'
    for pos in used_poses:
        cnt = sum(gold == pos for gold in golds)
        print(f'{pos}: {frac_format(cnt, len(golds))}')
    print(f'accuracy: {frac_format(sum(corrects), len(corrects))}')

load cached pos tags: dataset_cache/saycam/val.pos.cache
we should ____ on some lights , huh ?
29.196 turn     | 26.931 put      29.196 turn     31.202 try      31.764 go       32.473 sit     
  8.8% turn     |  84.7% put        8.8% turn       1.2% try        0.7% go         0.3% sit     
noun    0.005862
verb    0.994138
dtype: float32
gold = 'verb', pred = 'verb', correct = True
we should turn on some ______ , huh ?
29.965 lights   | 28.359 music    29.216 toys     29.219 light    29.362 water    29.588 things  
  4.0% lights   |  19.9% music      8.4% toys       8.4% light      7.3% water      5.8% things  
noun    0.978101
verb    0.021899
dtype: float32
gold = 'noun', pred = 'noun', correct = True
here , ___ me show you .
18.158 let      | 18.158 let      23.580 help     24.973 watch    25.489 make     26.363 give    
 99.2% let      |  99.2% let        0.4% help       0.1% watch      0.1% make       0.0% give    
noun    0.000394
verb    0.999606
dtype: float32
gold = 'verb', pr

83.755 go       | 83.755 go       86.862 have     89.020 come     90.389 are      90.749 went    
 94.7% go       |  94.7% go         4.2% have       0.5% come       0.1% are        0.1% went    
noun    0.000593
verb    0.999407
dtype: float32
gold = 'verb', pred = 'verb', correct = True
there we go , now can you ____ me , did you want to help me pick out some clothes look we have new
85.446 help     | 79.230 show     80.082 bring    80.096 come     80.597 have     81.046 get     
  0.1% help     |  34.8% show      14.8% bring     14.6% come       8.9% have       5.7% get     
noun    0.022005
verb    0.977995
dtype: float32
gold = 'verb', pred = 'verb', correct = True
there we go , now can you help me , ___ you want to help me pick out some clothes look we have new
81.175 did      | 73.668 do       80.136 get      81.175 did      81.761 have     82.868 come    
  0.1% did      |  99.7% do         0.2% get        0.1% did        0.0% have       0.0% come    
noun    0.000181
verb    0

44.752 sunscreen | 40.739 water    40.800 oil      40.807 sand     41.253 put      41.395 sheets  
  0.2% sunscreen |   9.6% water      9.0% oil        8.9% sand       5.7% put        5.0% sheets  
noun    0.864088
verb    0.135912
dtype: float32
gold = 'noun', pred = 'noun', correct = True
i am going put more sunscreen on this ___ , okay ?
43.011 arm      | 40.776 time     41.036 water    41.244 way      41.259 one      41.731 chair   
  1.7% arm      |  15.5% time      11.9% water      9.7% way        9.5% one        6.0% chair   
noun    0.97152
verb    0.02848
dtype: float32
gold = 'noun', pred = 'noun', correct = True
which ___ do you want ?
 8.735 one      |  8.735 one      10.709 socks    10.796 book     10.821 ones     11.648 pants   
 57.1% one      |  57.1% one        7.9% socks      7.3% book       7.1% ones       3.1% pants   
noun    0.997045
verb    0.002954
dtype: float32
gold = 'noun', pred = 'noun', correct = True
which one __ you want ?
 8.679 do       |  8.679 do    

28.309 's       | 28.309 's       31.598 ’s       33.124 s        33.996 is       36.056 're     
 95.3% 's       |  95.3% 's         3.6% ’s         0.8% s          0.3% is         0.0% 're     
noun    0.000020
verb    0.999981
dtype: float32
gold = 'verb', pred = 'verb', correct = True
op that 's not exactly how it __ .
29.319 is       | 28.579 works    29.098 did      29.319 is       29.523 does     29.741 's      
 11.8% is       |  24.8% works     14.8% did       11.8% is         9.7% does       7.8% 's      
noun    0.02985
verb    0.97015
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you ___ me .
17.813 pat      | 14.156 watch    14.810 want     15.005 thank    15.109 help     15.261 grab    
  0.4% pat      |  14.5% watch      7.5% want       6.2% thank      5.6% help       4.8% grab    
noun    0.030996
verb    0.969004
dtype: float32
gold = 'verb', pred = 'verb', correct = True
that __ good enough .
12.216 's       | 12.216 's       14.142 was      16.214 looks

 91.7% remember |  91.7% remember   3.9% know       3.3% want       0.4% see        0.3% think   
noun    0.000082
verb    0.999918
dtype: float32
gold = 'verb', pred = 'verb', correct = True
" do you remember how to ____ it on ?
17.562 turn     | 17.562 turn     19.935 put      23.717 make     23.750 take     23.969 keep    
 90.1% turn     |  90.1% turn       8.4% put        0.2% make       0.2% take       0.1% keep    
noun    0.001233
verb    0.998768
dtype: float32
gold = 'verb', pred = 'verb', correct = True
but , sometimes i ______ and slip and sprawl . "
25.380 wobble   | 25.380 wobble   32.247 stumble  35.352 poos     35.722 tried    36.111 twist   
 99.9% wobble   |  99.9% wobble     0.1% stumble    0.0% poos       0.0% tried      0.0% twist   
noun    0.000075
verb    0.999926
dtype: float32
gold = 'verb', pred = 'verb', correct = True
but , sometimes i wobble and ____ and sprawl . "
28.539 slip     | 28.539 slip     29.111 stretch  29.787 sprawl   31.002 roll     31.231 cho

...

33.901 georgia  | 27.961 something 29.735 one      30.008 johnny   30.396 kitty    30.806 bear    
  0.1% georgia  |  42.0% something   7.1% one        5.4% johnny     3.7% kitty      2.4% bear    
noun    0.89543
verb    0.10457
dtype: float32
gold = 'noun', pred = 'noun', correct = True
___ out of order , but -- and it 's a different because we 're not going to the store .
82.450 bit      | 75.601 made     75.936 pick     76.074 make     76.634 looks    77.029 get     
  0.0% bit      |  22.1% made      15.8% pick      13.8% make       7.9% looks      5.3% get     
noun    0.1541
verb    0.8459
dtype: float32
gold = 'noun', pred = 'verb', correct = False
bit out of _____ , but -- and it 's a different because we 're not going to the store .
81.456 order    | 74.151 things   75.547 course   75.683 birds    76.353 grapes   76.535 dots    
  0.0% order    |  35.3% things     8.7% course     7.6% birds      3.9% grapes     3.3% dots    
noun    0.960253
verb    0.039747
dtype: float32
go

19.329 are      | 17.689 're      19.329 are      21.590 ’re      23.304 's       23.445 re      
 15.7% are      |  81.1% 're       15.7% are        1.6% ’re        0.3% 's         0.3% re      
noun    0.000183
verb    0.999817
dtype: float32
gold = 'verb', pred = 'verb', correct = True
we are _____ to play here .
20.152 going    | 20.152 going    23.801 supposed 24.104 got      24.556 trying   24.844 want    
 90.8% going    |  90.8% going      2.4% supposed   1.7% got        1.1% trying     0.8% want    
noun    0.007234
verb    0.992766
dtype: float32
gold = 'verb', pred = 'verb', correct = True
we are going to ____ here .
20.572 play     | 18.086 go       19.049 walk     19.214 stay     19.875 sit      20.152 sleep   
  2.5% play     |  30.0% go        11.5% walk       9.7% stay       5.0% sit        3.8% sleep   
noun    0.068129
verb    0.931871
dtype: float32
gold = 'verb', pred = 'verb', correct = True
here we __ , yeah .
12.166 go       | 12.166 go       15.684 are      18.2

29.781 going    | 29.781 going    33.023 have     33.695 supposed 35.517 trying   36.080 need    
 93.5% going    |  93.5% going      3.7% have       1.9% supposed   0.3% trying     0.2% need    
noun    0.00138
verb    0.99862
dtype: float32
gold = 'verb', pred = 'verb', correct = True
we 're not going to ____ it either .
30.623 open     | 23.519 have     23.765 do       23.791 make     24.184 see      24.822 carry   
  0.0% open     |  13.7% have      10.7% do        10.4% make       7.0% see        3.7% carry   
noun    0.087868
verb    0.912132
dtype: float32
gold = 'verb', pred = 'verb', correct = True
" all what you _____ "
34.432 sound    | 25.872 do       26.359 did      26.831 want     26.944 're      26.984 think   
  0.0% sound    |  25.0% do        15.4% did        9.6% want       8.6% 're        8.2% think   
noun    0.018467
verb    0.981533
dtype: float32
gold = 'verb', pred = 'verb', correct = True
can i ______ it then ?
24.381 finish   | 20.299 do       20.706 have    

39.721 be       | 35.810 get      37.216 getting  37.262 see      37.639 sit      38.176 are     
  0.8% be       |  42.1% get       10.3% getting    9.9% see        6.8% sit        4.0% are     
noun    0.01181
verb    0.98819
dtype: float32
gold = 'verb', pred = 'verb', correct = True
hopefully this __ enough in here otherwise i have to warm up some more water .
76.141 is       | 74.647 has      76.141 is       76.318 's       78.164 needs    78.462 tastes  
 13.9% is       |  62.1% has       13.9% is        11.7% 's         1.8% needs      1.4% tastes  
noun    0.021654
verb    0.978346
dtype: float32
gold = 'verb', pred = 'verb', correct = True
hopefully this is enough in here otherwise i ____ to warm up some more water .
75.807 have     | 70.227 want     74.896 forgot   75.733 used     75.807 have     75.943 need    
  0.4% have     |  97.2% want       0.9% forgot     0.4% used       0.4% have       0.3% need    
noun    0.000283
verb    0.999717
dtype: float32
gold = 'verb', pred

20.215 pat      | 11.426 see      12.478 find     14.013 hear     14.149 get      15.111 feel    
  0.0% pat      |  59.3% see       20.7% find       4.5% hear       3.9% get        1.5% feel    
noun    0.005388
verb    0.994612
dtype: float32
gold = 'verb', pred = 'verb', correct = True
can you pat the _____ ?
20.887 kitty    | 13.144 sofa     14.702 ball     15.998 egg      16.748 water    16.776 grass   
  0.0% kitty    |  58.0% sofa      12.2% ball       3.3% egg        1.6% water      1.5% grass   
noun    0.982597
verb    0.017403
dtype: float32
gold = 'noun', pred = 'noun', correct = True
i __ n't need to fall and hurt your butt .
40.826 do       | 40.826 do       47.314 does     47.581 did      47.652 have     48.221 think   
 99.5% do       |  99.5% do         0.2% does       0.1% did        0.1% have       0.1% think   
noun    0.000146
verb    0.999853
dtype: float32
gold = 'verb', pred = 'verb', correct = True
i do n't ____ to fall and hurt your butt .
43.607 need     | 40